# Bike

В соревновании используется метрика среднеквадратичной логарифмической ошибки (RMSLE)

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor 

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_log_error

Загрузка данных

In [2]:
train_df = pd.read_csv('D:/Kode/MO/train.csv')
test_df = pd.read_csv('D:/Kode/MO/test.csv')

In [3]:
train_df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [4]:
test_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


Отделение целевой переменной

In [5]:
y_train = train_df['count']

In [6]:
train_df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [7]:
test_df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
count,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000
mean,2.493300,0.029108,0.685815,1.436778,20.620607,24.012865,64.125212,12.631157
std,1.091258,0.168123,0.464226,0.648390,8.059583,8.782741,19.293391,8.250151
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.000000,16.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,49.000000,7.001500
50%,3.000000,0.000000,1.000000,1.000000,21.320000,25.000000,65.000000,11.001400
75%,3.000000,0.000000,1.000000,2.000000,27.060000,31.060000,81.000000,16.997900
max,4.000000,1.000000,1.000000,4.000000,40.180000,50.000000,100.000000,55.998600


Преобразование элементов столбца 'datetime' в формат datetime

In [8]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])

In [9]:
train_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB


Вычленение из столбца 'datetime' в отдельные столбцы месяц, час и день недели

In [10]:
train_df['month'] = train_df['datetime'].apply(lambda t: t.month)
train_df['hour'] = train_df['datetime'].apply(lambda t: t.hour)
train_df['name_day'] = train_df['datetime'].apply(lambda t: t.dayofweek)

test_df['month'] = test_df['datetime'].apply(lambda t: t.month)
test_df['hour'] = test_df['datetime'].apply(lambda t: t.hour)
test_df['name_day'] = test_df['datetime'].apply(lambda t: t.dayofweek)

Перетасуем столбцы так, чтоб столбцы, производные от 'datetime', находились рядом для красоты

In [11]:
train_df = train_df[['datetime', 'month', 'hour', 'name_day', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']]
test_df = test_df[['datetime', 'month', 'hour', 'name_day', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]

In [12]:
train_df

,datetime,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,5,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,1,5,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,2,5,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,3,5,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,4,5,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,12,19,2,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,12,20,2,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,12,21,2,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,12,22,2,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [13]:
train_df.describe()

,datetime,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2011-12-27 05:56:22.399411968,6.521495,11.541613,3.013963,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
min,2011-01-01 00:00:00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2011-07-02 07:15:00,4.000000,6.000000,1.000000,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,2012-01-01 20:30:00,7.000000,12.000000,3.000000,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,2012-07-01 12:45:00,10.000000,18.000000,5.000000,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,2012-12-19 23:00:00,12.000000,23.000000,6.000000,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000
std,NaN,3.444373,6.915838,2.004585,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454


## Дерево решений

Удаление из train_df casual, registered и count, т.к. их нет в тестовых данных

In [99]:
train_df_no_dt = train_df.drop('casual', axis=1).drop('registered', axis=1).drop('count', axis=1)

train_df_no_dt = train_df_no_dt.set_index('datetime')
test_df_no_dt = test_df.set_index('datetime')

In [100]:
train_df_no_dt.head()

,month,hour,name_day,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,5,1,0,0,1,9.84,14.395,81,0.0
2011-01-01 01:00:00,1,1,5,1,0,0,1,9.02,13.635,80,0.0
2011-01-01 02:00:00,1,2,5,1,0,0,1,9.02,13.635,80,0.0
2011-01-01 03:00:00,1,3,5,1,0,0,1,9.84,14.395,75,0.0
2011-01-01 04:00:00,1,4,5,1,0,0,1,9.84,14.395,75,0.0


Кодирование категориальных признаков и добавление их в DataFrame

In [17]:
train_df_dummies = pd.concat([#pd.get_dummies(train_df['name_day'], prefix="name_day"),
                              pd.get_dummies(train_df['season'], prefix="season"),
                              pd.get_dummies(train_df['weather'], prefix="weather")],
                              #pd.get_dummies(train_df['holiday'], prefix="holiday"),
                              #pd.get_dummies(train_df['workingday'], prefix="workingday")],
                              axis=1)

test_df_dummies = pd.concat([#pd.get_dummies(test_df['name_day'], prefix="name_day"),
                              pd.get_dummies(test_df['season'], prefix="season"),
                              pd.get_dummies(test_df['weather'], prefix="weather")],
                              #pd.get_dummies(test_df['holiday'], prefix="holiday"),
                              #pd.get_dummies(test_df['workingday'], prefix="workingday")],
                              axis=1)

Добавление оставшихся признаков

In [18]:
train_df_dummies = pd.concat([train_df_dummies,
                              train_df['month'],
                              train_df['hour'],
                              train_df['temp'],
                              train_df['atemp'],
                              train_df['humidity'],
                              train_df['windspeed'],
                              train_df['name_day'],
                              train_df['holiday'],
                              train_df['workingday']],
                              axis=1)

test_df_dummies = pd.concat([test_df_dummies,
                              test_df['month'],
                              test_df['hour'],
                              test_df['temp'],
                              test_df['atemp'],
                              test_df['humidity'],
                              test_df['windspeed'],
                              train_df['name_day'],
                              train_df['holiday'],
                              train_df['workingday']],
                              axis=1)

In [19]:
train_df_dummies.head()

,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,month,hour,temp,atemp,humidity,windspeed,name_day,holiday,workingday
0,True,False,False,False,True,False,False,False,1,0,9.84,14.395,81,0.0,5,0,0
1,True,False,False,False,True,False,False,False,1,1,9.02,13.635,80,0.0,5,0,0
2,True,False,False,False,True,False,False,False,1,2,9.02,13.635,80,0.0,5,0,0
3,True,False,False,False,True,False,False,False,1,3,9.84,14.395,75,0.0,5,0,0
4,True,False,False,False,True,False,False,False,1,4,9.84,14.395,75,0.0,5,0,0


In [18]:
train_df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name_day_0    10886 non-null  bool   
 1   name_day_1    10886 non-null  bool   
 2   name_day_2    10886 non-null  bool   
 3   name_day_3    10886 non-null  bool   
 4   name_day_4    10886 non-null  bool   
 5   name_day_5    10886 non-null  bool   
 6   name_day_6    10886 non-null  bool   
 7   season_1      10886 non-null  bool   
 8   season_2      10886 non-null  bool   
 9   season_3      10886 non-null  bool   
 10  season_4      10886 non-null  bool   
 11  weather_1     10886 non-null  bool   
 12  weather_2     10886 non-null  bool   
 13  weather_3     10886 non-null  bool   
 14  weather_4     10886 non-null  bool   
 15  holiday_0     10886 non-null  bool   
 16  holiday_1     10886 non-null  bool   
 17  workingday_0  10886 non-null  bool   
 18  workingday_1  10886 non-nu

Функция для оценки качества модели дерева решений

In [50]:
def score_d_tree(X, y, depth = 2, leaf=1, ratio=0.8, seed=17):
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    d_tree = DecisionTreeRegressor(max_depth=depth, min_samples_leaf=leaf, random_state=seed)
    d_tree.fit(x_train, y_train)

    #pred = d_tree.predict(x_test)

    scored = d_tree.score(x_test, y_test)

    #get_tree_image_3(d_tree)
    
    return scored

### Обучение без настройки параметров дерева

- кодированные данные

In [51]:
score_d_tree(train_df_no_dt, y_train)

0.41537957681133775

- не кодированные данные

In [52]:
score_d_tree(train_df_dummies, y_train)

0.41537957681133775

### Нахождение лучших параметров дерева

In [56]:
def find_best_tree_params(X, y, tree_params={}, ratio=0.8, seed=17):
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    tree = DecisionTreeRegressor(max_depth=2, random_state=seed)

    tree_grid = GridSearchCV(tree, tree_params,
                            cv=5, n_jobs=-1,
                            verbose=True)

    tree_grid.fit(x_train, y_train)

    #pred = tree_grid.predict(x_test)
    #rmsle = mean_squared_log_error(y_test, pred)
    score = tree_grid.score(x_test, y_test)
    
    return score, tree_grid.best_params_

- не кодированные данные

In [57]:
tree_params = {'max_depth': list(range(1, 20)), 
               'min_samples_leaf': list(range(1, 20))}
rmsle_best_params_1, best_params_1 = find_best_tree_params(train_df_no_dt, y_train, tree_params, ratio=0.8)
print(rmsle_best_params_1, '\n',
      best_params_1)

Fitting 5 folds for each of 361 candidates, totalling 1805 fits
0.8209772892584368 
 {'max_depth': 12, 'min_samples_leaf': 9}


- кодированные данные

In [58]:
tree_params = {'max_depth': list(range(1, 20)), 
               'min_samples_leaf': list(range(1, 20))}
rmsle_best_params_2, best_params_2 = find_best_tree_params(train_df_dummies, y_train, tree_params, ratio=0.8)
print(rmsle_best_params_2, '\n',
      best_params_2)

Fitting 5 folds for each of 361 candidates, totalling 1805 fits
0.8204932596128576 
 {'max_depth': 12, 'min_samples_leaf': 11}


In [59]:
rmsle_best_params_2 - rmsle_best_params_1

-0.00048402964557925365

### Предсказание на тестовых данных

Функция для записи прогноза в файл

In [128]:
def write_to_submission_file(predicted_labels, out_file,
                             target='count', index_label='datetime'):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = test_df[index_label],
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [119]:
tree = DecisionTreeRegressor(max_depth=2, random_state=17)

tree_grid_1 = GridSearchCV(tree, tree_params,
                        cv=5, n_jobs=-1,
                        verbose=True)

tree_grid_1.fit(train_df_no_dt, y_train)

pred_1 = tree_grid_1.predict(test_df_no_dt)
print('best params:', tree_grid_1.best_params_)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [28]:
tree = DecisionTreeRegressor(max_depth=2, random_state=17)

tree_grid_2 = GridSearchCV(tree, tree_params,
                        cv=5, n_jobs=-1,
                        verbose=True)

tree_grid_2.fit(train_df_dummies, y_train)

pred_2 = tree_grid_2.predict(test_df_dummies)
print('best params:', tree_grid_2.best_params_)

Fitting 5 folds for each of 361 candidates, totalling 1805 fits
best params: {'max_depth': 9, 'min_samples_leaf': 16}


In [114]:
write_to_submission_file(pred_1, 'dtree_1.csv') # 52%

In [108]:
write_to_submission_file(pred_2, 'dtree_2.csv') # 52%

## Random Forest

Без настройки

In [20]:
def rforest_valid(X, y, ratio = 0.8, seed=17):
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    random_tree = RandomForestRegressor()
    random_tree.fit(x_train, y_train)

    score = random_tree.score(x_test, y_test)
    return score

- не кодированные данные

In [67]:
rforest_score_1 = rforest_valid(train_df_no_dt, y_train)
rforest_score_1

0.8713900709287289

- кодированные данные

In [49]:
rtree_score_1_k = rforest_valid(train_df_dummies, y_train)

0.8742379363902202

In [21]:
def find_best_params_rforest(X, y, rtree_params, ratio = 0.8, seed=17):
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    rforest = RandomForestRegressor()
    
    rforest_grid = GridSearchCV(rforest, rtree_params,
                                    cv=5, n_jobs=-1,
                                    verbose=True)

    rforest_grid.fit(x_train, y_train)

    score = rforest_grid.score(x_test, y_test)
    return score, rforest_grid.best_params_

In [69]:
rforest_params = {'max_depth': list(range(1, 20)), 
               'min_samples_leaf': list(range(1, 20))}
rforest_score_2, rtreebest_params_2 = find_best_params_rforest(train_df_no_dt, y_train, rforest_params)
print(rforest_score_2, '\n',
      best_params_2)

0.8730455793371021 
 {'max_depth': 12, 'min_samples_leaf': 11}


In [68]:
rforest_score_2 - rforest_score_1

0.0016555084083732519

Поиск лучших параметров по одному:

In [24]:
from sklearn.metrics import make_scorer

rmsle_scorer = make_scorer(mean_squared_log_error, greater_is_better=False)

- n_estimators

In [56]:
rforest_params = {'n_estimators': [50, 100, 200, 500, 1000]}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

KeyboardInterrupt: 

- max_features

In [55]:
rforest_params = {'n_estimators': [200], 'max_features': range(5, 15, 2)}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

{'max_features': 9, 'n_estimators': 200} 
 rmsle:  0.03170935261704291


- min_samples_leaf

In [53]:
rforest_params = {'n_estimators': [200], 'max_features': [9], 'min_samples_leaf':range(1, 10)}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

{'max_depth': 40, 'max_features': 9, 'min_samples_leaf': 2, 'n_estimators': 200} 
 rmsle:  0.053797544156094106


- max_depth

In [52]:
rforest_params = {'n_estimators': [200], 'max_features': [9], 'min_samples_leaf': [2], 'max_depth':[5, 8, 10, 20, 30, 40, 50, 70]}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

{'max_depth': 40, 'max_features': 9, 'min_samples_leaf': 2, 'n_estimators': 200} 
 rmsle:  0.05340671400620412


In [ ]:
rforest_params = {'n_estimators': [200], 'max_features': [9], 'min_samples_leaf': [2], 'max_depth':[5, 8, 10, 20, 30, 40, 50, 70]}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

In [54]:
rforest_params = {'n_estimators': [200], 'max_features': [9], 'min_samples_leaf': [2], 'max_depth':[40], 'min_samples_split':[0.0001, 0.001, 0.002, 0.005, 0.01]}

rforest_grid = GridSearchCV(RandomForestRegressor(), rforest_params, cv=5, scoring=rmsle_scorer, n_jobs=-1)
rforest_grid.fit(train_df_no_dt, y_train)

rforest_pred_grid = rforest_grid.predict(train_df_no_dt)
rmsle_rforest_grid = mean_squared_log_error(y_true=y_train, y_pred=rforest_pred_grid)

print(rforest_grid.best_params_, '\n'
      ,'rmsle: ',rmsle_rforest_grid)

{'max_depth': 40, 'max_features': 9, 'min_samples_leaf': 2, 'min_samples_split': 0.0001, 'n_estimators': 200} 
 rmsle:  0.053651669709953886


Предсказание test_df

In [65]:
rforest_est = RandomForestRegressor(n_estimators=200)
rforest_est.fit(train_df_no_dt, y_train)

rforest_pred_grid_est = rforest_est.predict(test_df_no_dt)

In [69]:
write_to_submission_file(rforest_pred_grid_est, 'rforest_est_1.csv')

In [101]:
rforest_all = RandomForestRegressor(max_depth=40, max_features=9, min_samples_leaf=2, min_samples_split=0.0001, n_estimators=200)
rforest_all.fit(train_df_no_dt, y_train)

rforest_pred_all = rforest_all.predict(test_df_no_dt)

In [102]:
write_to_submission_file(rforest_pred_all, 'rforest_all_1.csv')

In [91]:
rforest_stock = RandomForestRegressor()
rforest_stock.fit(train_df_no_dt, y_train)

rforest_pred_stock = rforest_stock.predict(test_df_no_dt)

In [73]:
write_to_submission_file(rforest_pred_stock, 'rforest_stock_1.csv')


Разделение данных на рабочие и не рабочие дни

In [86]:
train_df_working = train_df[train_df['workingday'] == 1]
train_df_no_working = train_df[train_df['workingday'] == 0]

y_train_working = train_df_working['count']
y_train_no_working = train_df_no_working['count']

train_df_working = train_df_working.drop('casual', axis=1).drop('registered', axis=1).drop('count', axis=1)
train_df_working = train_df_working.set_index('datetime')



train_df_no_working = train_df_no_working.drop('casual', axis=1).drop('registered', axis=1).drop('count', axis=1)
train_df_no_working = train_df_no_working.set_index('datetime')

test_df_working = test_df[test_df['workingday'] == 1]
test_df_no_working = test_df[test_df['workingday'] == 0]
test_df_working = test_df_working.set_index('datetime')
test_df_no_working = test_df_no_working.set_index('datetime')

In [104]:
rforest_working = RandomForestRegressor()
rforest_working.fit(train_df_working, y_train_working)

rforest_pred_working = rforest_working.predict(test_df_working)

In [105]:
rforest_no_working = RandomForestRegressor()
rforest_no_working.fit(train_df_no_working, y_train_no_working)

rforest_pred_no_working = rforest_no_working.predict(test_df_no_working)

In [110]:
predicted_df_working = pd.DataFrame(rforest_pred_working,
                                index = test_df_working.index,
                                columns=['count'])
predicted_df_no_working = pd.DataFrame(rforest_pred_no_working,
                                index = test_df_no_working.index,
                                columns=['count'])
predicted_df_ftree_split = pd.concat([predicted_df_working, predicted_df_no_working])

In [114]:
predicted_df_ftree_split.to_csv('rforest_split.csv', index_label='datetime')

Поиск зарегистрированных и не зарегистрированных пользователей по отдельности

In [116]:
y_reg_train = train_df['registered']
y_cas_train = train_df['casual']

In [134]:
rforest_reg = RandomForestRegressor()
rforest_reg.fit(train_df_no_dt, y_reg_train)

rforest_pred_reg = rforest_reg.predict(test_df_no_dt)

rforest_cas = RandomForestRegressor()
rforest_cas.fit(train_df_no_dt, y_cas_train)

rforest_pred_cas = rforest_cas.predict(test_df_no_dt)

In [135]:
write_to_submission_file(rforest_pred_reg + rforest_pred_cas, 'reg_cas.csv')